In [2]:
"""This script will clean your data/channels/coordinates,
adapt it from matlab format and save in a convenient
numpy format.
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
from os import path

from brainpipe.system import study

def _cleanData(pathdata, suj, phase,trigger):
    """The function to clean your data. This is convenient
    because you can loop on your subject/trigger and clean
    everything in one script.
    """

    # Import file :
    #file = '{suj}_R1R2R3_concat_{trigg}.mat'
    #file = '{suj}_{trigg}_R1R2R3_concat_all.mat'
    #file = '{suj}_{trigg}_E1E2_concat_all.mat'
    file = '{suj}_{phase}_{trigg}.mat'
    #file = '{suj}_E1E2_concat_{trigg}.mat'
    #file = '{suj}_E1E2_concat_{trigg}.mat'
    loadname = file.format(suj=suj, phase=phase,trigg=trigger)
    print('-> Processing '+loadname)
    fname = path.join(pathdata, loadname)
    #print(fname)
    mat = loadmat(fname)
    mat2 = mat['x']
    print (mat2.shape)

    # Make a clean version of channel name / xyz :
    chan_xy = mat['chanmonop'][1:4]
    print (chan_xy.shape)
    chan_names = mat['chanmonop'][0]
    chan_label = mat['chanmonop'][4]
    # channel = [k[0][0] for k in chan[0]]  # Clean channel name
    xyz = np.zeros([len(chan_names), 3], dtype=float)
    for k in range(len(chan_names)):
        xyz[k, 0] = float(chan_xy[0,k][0])
        xyz[k, 1] = float(chan_xy[1,k][0])
        xyz[k, 2] = float(chan_xy[2,k][0])
    xyz = xyz.round(decimals=1, out=None)
    #print (xyz)

    # Clean matrix of data (remove unecessary channels)
    x = mat['x']
    print ('shape of the data', x.shape)
    if x.ndim == 2:  # This test is in case of x being a 2D matrix (make it 3D)
        x = x[..., np.newaxis]
    x = x.swapaxes(0, 1).swapaxes(1,2)
    print (x.shape)

    # Update variable inside the loaded file :
    dico = dict(mat)
    dico['x'] = x[0:len(chan_names), ...]
    dico['xyz'] = (xyz)
    dico['channel'] = chan_names
    dico['label'] = chan_label
    dico['sf'] = dico['sf'][0][0]
    del dico['chanmonop']
    del dico['__header__']
    del dico['__globals__']
    del dico['__version__']

    return dico, loadname.split('.mat')[0]


if __name__ == '__main__':
    # Filesettings :
    st = study('Olfacto') # Load the study
    pathdata = r'/media/karim/Datas4To/TS_E_all_cond_by_block_trigs/mat_by_odor/' # path where data are located
    #suj = ['CHAF', 'VACJ', 'SEMC', 'PIRJ', 'LEFC', 'MICP','FERJ']
    suj = ['PIRJ']
    #trigger_to_calculate = ['rest', 'odor']
    #trigger_to_calculate = ['0', '1', '2']
    #trigger_to_calculate = ['early', 'late']
    #trigger_to_calculate = ['bad', 'good']
    phases = ['odor']

    trigger_to_calculate = {
       #'CHAF' : ['1','2','3','4','5','7','8','9'],
       #'VACJ' : ['14','15','16','17','10','11','12','13'], 
       #'SEMC' : ['5','7','8','9','10','11','12','13'],
       'PIRJ' : ['1','4','9','18','6','7','15','5'],
       #'LEFC' : ['1','2','3','4','14','15','16','17'],
       #'MICP' : ['3','9','14','18','2','6','8','12'],
               }

    for su in suj:
        for phase in phases:
            for tr in trigger_to_calculate[su]:
                print (su, phase,tr)
                dico, shortname = _cleanData(pathdata, su, phase,tr)
                path2save = path.join(st.path, 'database/Encoding_By_Odor/', shortname+'_new.npz') # make sure it's windows compatible
                np.savez(path2save, **dico)
            #print('-> Save as : '+path2save+'\n')


-> Olfacto loaded
PIRJ odor 1
-> Processing PIRJ_odor_1.mat
(10, 111, 3584)
(3, 111)
shape of the data (10, 111, 3584)
(111, 3584, 10)
PIRJ odor 4
-> Processing PIRJ_odor_4.mat
(10, 111, 3584)
(3, 111)
shape of the data (10, 111, 3584)
(111, 3584, 10)
PIRJ odor 9
-> Processing PIRJ_odor_9.mat
(10, 111, 3584)
(3, 111)
shape of the data (10, 111, 3584)
(111, 3584, 10)
PIRJ odor 18
-> Processing PIRJ_odor_18.mat
(10, 111, 3584)
(3, 111)
shape of the data (10, 111, 3584)
(111, 3584, 10)
PIRJ odor 6
-> Processing PIRJ_odor_6.mat
(10, 111, 3584)
(3, 111)
shape of the data (10, 111, 3584)
(111, 3584, 10)
PIRJ odor 7
-> Processing PIRJ_odor_7.mat
(10, 111, 3584)
(3, 111)
shape of the data (10, 111, 3584)
(111, 3584, 10)
PIRJ odor 15
-> Processing PIRJ_odor_15.mat
(11, 111, 3584)
(3, 111)
shape of the data (11, 111, 3584)
(111, 3584, 11)
PIRJ odor 5
-> Processing PIRJ_odor_5.mat
(10, 111, 3584)
(3, 111)
shape of the data (10, 111, 3584)
(111, 3584, 10)


In [28]:
file = '{suj}_{tr}_R1R2R3_concat_allfilter1_bipo.npz'
#file = '{suj}_R1R2R3_concat_{trigg}filter1_bipo.npz'
#file = '{suj}_E1E2_concat_allfilter1_bipo.npz'
pathdata = r'/media/karim/Datas4To/Analyses_Intra_EM_Odor/Olfacto/database/TS_R_Rec_cond_by_block_trigs_filter1_500art' # path where data are located
suj = ['CHAF', 'VACJ', 'SEMC', 'FERJ',]
trigg = ['new', 'old']
tr = {    
    'CHAF' :   ['cr', 'hit', 'miss',],
    'VACJ' :   ['cr', 'hit', 'miss',],
    'SEMC' :   ['cr', 'hit', 'miss', ],
    'FERJ' :   ['cr', 'hit', 'miss', 'fa'],
    }

for su in suj:
    for trigg in tr[su]:
    #for trig in trigg:
        loadname = file.format(suj=su, tr=trigg)
        #loadname = file.format(suj=su, trigg=trig)
        print (loadname)
        fname = path.join(pathdata, loadname)
        mat = np.load(fname)
        x = mat['x']
        print (x.shape)


CHAF_cr_R1R2R3_concat_allfilter1_bipo.npz
(108, 3072, 20)
CHAF_hit_R1R2R3_concat_allfilter1_bipo.npz
(108, 3072, 19)
CHAF_miss_R1R2R3_concat_allfilter1_bipo.npz
(108, 3072, 6)
VACJ_cr_R1R2R3_concat_allfilter1_bipo.npz
(138, 3072, 17)
VACJ_hit_R1R2R3_concat_allfilter1_bipo.npz
(138, 3072, 16)
VACJ_miss_R1R2R3_concat_allfilter1_bipo.npz
(138, 3072, 18)
SEMC_cr_R1R2R3_concat_allfilter1_bipo.npz
(107, 3072, 25)
SEMC_hit_R1R2R3_concat_allfilter1_bipo.npz
(107, 3072, 24)
SEMC_miss_R1R2R3_concat_allfilter1_bipo.npz
(107, 3072, 28)
FERJ_cr_R1R2R3_concat_allfilter1_bipo.npz
(110, 3072, 29)
FERJ_hit_R1R2R3_concat_allfilter1_bipo.npz
(110, 3072, 27)
FERJ_miss_R1R2R3_concat_allfilter1_bipo.npz
(110, 3072, 30)
FERJ_fa_R1R2R3_concat_allfilter1_bipo.npz
(110, 3072, 21)
